In [4]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image

In [2]:
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 15

TRAIN_DIR = "D:/New download/chest_xray\chest_xray/train"
VAL_DIR   = "D:/New download/chest_xray\chest_xray/val"
TEST_DIR  = "D:/New download/chest_xray\chest_xray/test"


In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, zoom_range=0.1, horizontal_flip=True)

val_test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode="binary")
val_generator = val_test_datagen.flow_from_directory(VAL_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode="binary")
test_generator = val_test_datagen.flow_from_directory(TEST_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode="binary", shuffle=False)
print("Class Mapping:", train_generator.class_indices)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Class Mapping: {'NORMAL': 0, 'PNEUMONIA': 1}


In [7]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

In [9]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])

In [10]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
early_stop = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath="pneumonia_model.keras", monitor="val_loss", save_best_only=True)

In [12]:
history = model.fit(train_generator, validation_data=val_generator,epochs=EPOCHS, callbacks=[early_stop, checkpoint])

c:\Users\gouth\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.7709 - loss: 0.4674 - val_accuracy: 1.0000 - val_loss: 0.2146
Epoch 2/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.8995 - loss: 0.2317 - val_accuracy: 0.9375 - val_loss: 0.2006
Epoch 3/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - accuracy: 0.9211 - loss: 0.1820 - val_accuracy: 0.9375 - val_loss: 0.1619
Epoch 4/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.9397 - loss: 0.1528 - val_accuracy: 0.9375 - val_loss: 0.1644
Epoch 5/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 143s 874ms/step - accuracy: 0.9430 - loss: 0.1558 - val_accuracy: 0.9375 - val_loss: 0.1603
Epoch 6/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 151s 927ms/step - accuracy: 0.9436 - loss: 0.1541 - val_accuracy: 1.0000 - val_loss: 0.1394
Epoch 7/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 158s 966ms/step - accuracy: 0.9486 - loss: 0.1326 - val_accuracy: 0.9375 - val_loss: 0.1403
Epoch 8/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.9497 - loss: 0.1320 -

In [13]:
model = load_model("pneumonia_model.keras")

In [14]:
test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test Loss     : {test_loss:.4f}")
print(f"Test Accuracy : {test_accuracy * 100:.2f}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.8794 - loss: 0.2842
Test Loss     : 0.2841
Test Accuracy : 87.66%


In [15]:
def predict_xray(img_path):
    img = image.load_img(img_path, target_size=IMAGE_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    if prediction >= 0.5:
        return f"PNEUMONIA (confidence: {prediction:.2f})"
    else:
        return f"NORMAL (confidence: {1 - prediction:.2f})"

In [16]:
sample_image = "D:/New download/chest_xray/chest_xray/train/PNEUMONIA/person10_bacteria_43.jpeg"
print(predict_xray(sample_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
PNEUMONIA (confidence: 0.94)


In [17]:
sample_image = "D:/New download/chest_xray/chest_xray/train/NORMAL/IM-0154-0001.jpeg"
print(predict_xray(sample_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
NORMAL (confidence: 1.00)


In [18]:
sample_image = "D:/New download/chest_xray/chest_xray/train/PNEUMONIA/person55_bacteria_260.jpeg"
print(predict_xray(sample_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
PNEUMONIA (confidence: 1.00)


In [19]:
sample_image = "D:/New download/chest_xray/chest_xray/train/NORMAL/IM-0684-0001.jpeg"
print(predict_xray(sample_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
NORMAL (confidence: 0.99)


In [21]:
sample_image = "D:/New download/76052f7902246ff862f52f5d3cd9cd_gallery.jpg"
print(predict_xray(sample_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
NORMAL (confidence: 0.63)
